In [1]:
import pandas as pd
import numpy as np
import os

In [105]:
dados = pd.read_csv('Campos.csv', sep=',')
dadosArray = np.array(dados)

flagsArray = []
datasArray = []
for i in dadosArray:
    flagsArray.append(i[0])
    datasArray.append(i[1])


In [106]:
teste = ''

for flag in flagsArray:
    teste += '\'' + flag + '\'' + ', '

teste


"'DS_REDUZIDA', 'IE_TIPO_MATERIAL', 'CD_UNIDADE_MEDIDA_COMPRA', 'CD_UNIDADE_MEDIDA_CONSUMO', 'CD_UNIDADE_MEDIDA_SOLIC', 'CD_MATERIAL_GENERICO', 'CD_MATERIAL_ESTOQUE', 'IE_VIA_APLICACAO', 'IE_SITUACAO', 'IE_OBRIG_VIA_APLICACAO', 'IE_RECEITA', 'IE_COBRA_PACIENTE', 'IE_INF_ULTIMA_COMPRA', 'IE_CURVA_ABC', 'NR_SEQ_FABRIC', 'QT_CONV_ESTOQUE_CONSUMO', 'QT_MINIMO_MULTIPLO_SOLIC', 'QT_PRIORIDADE_COML', 'NR_SEQ_FAMILIA', 'CD_MATERIAL_CONTA', 'IE_MATERIAL_ESTOQUE', 'IE_BAIXA_ESTOQ_PAC', 'IE_PADRONIZADO', 'IE_CURVA_XYZ', 'NR_REGISTRO_ANVISA', 'DT_VALIDADE_REG_ANVISA', 'IE_VIGENTE_ANVISA', 'IE_REG_ANVISA_ISENTO', 'DS_MOTIVO_ISENCAO_ANVISA', 'IE_UNID_CONSUMO_PRESCR', 'NR_SEQ_FICHA_TECNICA', 'IE_ALTO_RISCO', 'IE_MULTIDOSE', 'CD_MEDICAMENTO', 'IE_SOLUCAO', 'IE_CONTROLE_MEDICO', 'CD_UNIDADE_MEDIDA', 'IE_TEMPO_ESTAB', 'IE_REGRA', "

In [107]:
tabelas = []
for flag in flagsArray:
    tabelas.append('flag_' + flag.lower())
tabelas

['flag_ds_reduzida',
 'flag_ie_tipo_material',
 'flag_cd_unidade_medida_compra',
 'flag_cd_unidade_medida_consumo',
 'flag_cd_unidade_medida_solic',
 'flag_cd_material_generico',
 'flag_cd_material_estoque',
 'flag_ie_via_aplicacao',
 'flag_ie_situacao',
 'flag_ie_obrig_via_aplicacao',
 'flag_ie_receita',
 'flag_ie_cobra_paciente',
 'flag_ie_inf_ultima_compra',
 'flag_ie_curva_abc',
 'flag_nr_seq_fabric',
 'flag_qt_conv_estoque_consumo',
 'flag_qt_minimo_multiplo_solic',
 'flag_qt_prioridade_coml',
 'flag_nr_seq_familia',
 'flag_cd_material_conta',
 'flag_ie_material_estoque',
 'flag_ie_baixa_estoq_pac',
 'flag_ie_padronizado',
 'flag_ie_curva_xyz',
 'flag_nr_registro_anvisa',
 'flag_dt_validade_reg_anvisa',
 'flag_ie_vigente_anvisa',
 'flag_ie_reg_anvisa_isento',
 'flag_ds_motivo_isencao_anvisa',
 'flag_ie_unid_consumo_prescr',
 'flag_nr_seq_ficha_tecnica',
 'flag_ie_alto_risco',
 'flag_ie_multidose',
 'flag_cd_medicamento',
 'flag_ie_solucao',
 'flag_ie_controle_medico',
 'flag_cd_un

In [108]:
datasTabelas = []
for data in datasArray:
    datasTabelas.append(data.lower())
datasTabelas

['dt_san_ds_reduzida',
 'dt_san_ie_tipo_material',
 'dt_san_cd_unidade_medida_compra',
 'dt_san_cd_unidade_medida_consumo',
 'dt_san_cd_unidade_medida_solic',
 'dt_san_cd_material_generico',
 'dt_san_cd_material_estoque',
 'dt_san_ie_via_aplicacao',
 'dt_san_ie_situacao',
 'dt_san_ie_obrig_via_aplicacao',
 'dt_san_ie_receita',
 'dt_san_ie_cobra_paciente',
 'dt_san_ie_inf_ultima_compra',
 'dt_san_ie_curva_abc',
 'dt_san_nr_seq_fabric',
 'dt_san_qt_conv_estoque_consumo',
 'dt_san_qt_minimo_multiplo_solic',
 'dt_san_qt_prioridade_coml',
 'dt_san_nr_seq_familia',
 'dt_san_cd_material_conta',
 'dt_san_ie_material_estoque',
 'dt_san_ie_baixa_estoq_pac',
 'dt_san_ie_padronizado',
 'dt_san_ie_curva_xyz',
 'dt_san_nr_registro_anvisa',
 'dt_san_dt_san_validade_reg_anvisa',
 'dt_san_ie_vigente_anvisa',
 'dt_san_ie_reg_anvisa_isento',
 'dt_san_ds_motivo_isencao_anvisa',
 'dt_san_ie_unid_consumo_prescr',
 'dt_san_nr_seq_ficha_tecnica',
 'dt_san_ie_alto_risco',
 'dt_san_ie_multidose',
 'dt_san_cd_me

In [109]:
def setLinha(flag, data, dado):
    return f'UPDATE \n    analytics.gcm_saneamento\nSET\n   flag_tipo_saneamento = CASE WHEN saneamento_materiais.{flag} = \'Sim\' \n                              AND saneamento_materiais.{data}::DATE <= gcm_saneamento.dt_alteracao::DATE\n                              AND gcm_saneamento.flag_tipo = \'Carga\' THEN \'Saneamento em Produção\'\n                         END\n, flag_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.{flag} = \'Sim\'\n                                      AND saneamento_materiais.{data}::DATE < gcm_saneamento.dt_alteracao::DATE\n                                      AND gcm_saneamento.flag_tipo NOT IN (\'Carga\') THEN \'Sim\'\n                                     ELSE \'Não\'\n                               END\n, qtd_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.{flag} = \'Sim\'\n                                   AND saneamento_materiais.{data}::DATE < gcm_saneamento.dt_alteracao::DATE\n                                   AND gcm_saneamento.flag_tipo NOT IN (\'Carga\') THEN 1\n                                   ELSE 0\n                              END\nFROM\n    ingestion.saneamento_materiais\nWHERE\n    saneamento_materiais.cd_material = gcm_saneamento.cd_material\n    AND gcm_saneamento.ds_campo = \'{dado}\'\n    AND gcm_saneamento.ds_tabela = \'MATERIAL\'\n;\n'

print(setLinha('flag_', 'dt_', 'DADO_'))

UPDATE 
    analytics.gcm_saneamento
SET
   flag_tipo_saneamento = CASE WHEN saneamento_materiais.flag_ = 'Sim' 
                              AND saneamento_materiais.dt_::DATE <= gcm_saneamento.dt_alteracao::DATE
                              AND gcm_saneamento.flag_tipo = 'Carga' THEN 'Saneamento em Produção'
                         END
, flag_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_ = 'Sim'
                                      AND saneamento_materiais.dt_::DATE < gcm_saneamento.dt_alteracao::DATE
                                      AND gcm_saneamento.flag_tipo NOT IN ('Carga') THEN 'Sim'
                                     ELSE 'Não'
                               END
, qtd_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_ = 'Sim'
                                   AND saneamento_materiais.dt_::DATE < gcm_saneamento.dt_alteracao::DATE
                                   AND gcm_saneamento.flag_tipo NOT IN ('Carga') THEN 1
                         

In [118]:
if os.path.exists("script.txt"):
    os.remove("script.txt")

texto = open('script.txt', 'a')
qtde = 0

for i in range(len(flagsArray)):
    texto.write('\n' + setLinha(tabelas[i], datasTabelas[i],flagsArray[i]))
    if i == 33:
        print(setLinha(tabelas[i], datasTabelas[i],flagsArray[i]))
texto.close()


UPDATE 
    analytics.gcm_saneamento
SET
   flag_tipo_saneamento = CASE WHEN saneamento_materiais.flag_cd_medicamento = 'Sim' 
                              AND saneamento_materiais.dt_san_cd_medicamento::DATE <= gcm_saneamento.dt_alteracao::DATE
                              AND gcm_saneamento.flag_tipo = 'Carga' THEN 'Saneamento em Produção'
                         END
, flag_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_cd_medicamento = 'Sim'
                                      AND saneamento_materiais.dt_san_cd_medicamento::DATE < gcm_saneamento.dt_alteracao::DATE
                                      AND gcm_saneamento.flag_tipo NOT IN ('Carga') THEN 'Sim'
                                     ELSE 'Não'
                               END
, qtd_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_cd_medicamento = 'Sim'
                                   AND saneamento_materiais.dt_san_cd_medicamento::DATE < gcm_saneamento.dt_alteracao::DATE
                 

In [112]:
setLinha(tabelas[36], datasTabelas[36], flagsArray[36])



"UPDATE \n    analytics.gcm_saneamento\nSET\n   flag_tipo_saneamento = CASE WHEN saneamento_materiais.flag_cd_unidade_medida = 'Sim' \n                              AND saneamento_materiais.dt_san_cd_unidade_medida::DATE <= gcm_saneamento.dt_alteracao::DATE\n                              AND gcm_saneamento.flag_tipo = 'Carga' THEN 'Saneamento em Produção'\n                         END\n, flag_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_cd_unidade_medida = 'Sim'\n                                      AND saneamento_materiais.dt_san_cd_unidade_medida::DATE < gcm_saneamento.dt_alteracao::DATE\n                                      AND gcm_saneamento.flag_tipo NOT IN ('Carga') THEN 'Sim'\n                                     ELSE 'Não'\n                               END\n, qtd_ajuste_pos_saneamento = CASE WHEN saneamento_materiais.flag_cd_unidade_medida = 'Sim'\n                                   AND saneamento_materiais.dt_san_cd_unidade_medida::DATE < gcm_saneamento.dt_a